In [1]:
#Importing necessary libraries
import pandas as pd 
import glob
import time
from random import randint
from time import sleep

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# selenium error handling
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException

In [14]:
# open website by webdriver
def openWebsite():
    
    global driver
    driver = webdriver.Chrome('C:/chromedriver-win64/chromedriver.exe')  # Optional argument, if not specified will search path. /path/to/chromedriver

    driver.maximize_window() #maximize window size
    driver.get('https://tmsearch.uspto.gov')
    # make sure searchbar loaded
    try:
        ele = WebDriverWait(driver, 10).until( #using explicit wait for 10 seconds
            EC.presence_of_element_located((By.ID, "searchbar")) 
        )
        print("Page is loaded within 10 seconds.")
    except:
        print("Timeout Exception: Page did not load within 10 seconds.")
    sleep(randint(0,3))


In [ ]:
#    close feedback box 
def closeFeedbackWindow():
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button//img[contains(@src,"wr-dialog-close-btn-black.png")][@alt="Close"]'))
        ).click()
        print("           ==>>> feedback window closed")
    except NoSuchElementException as ne:
        print(ne)
        print("  no feedback window fund")
    except TimeoutException as te:
        print(te)
        print("  timeout for feedback window to be clickable")   
    except ElementClickInterceptedException as interceptedE:
        print(interceptedE)
        print("  timeout for feedback window to be clickable")
    except(Exception):
        print("   Exception: feedback window to be clickable")
        print(Exception)

In [3]:
# Read and store content 
# of an excel file  C:\Users\Administrator\Desktop
df = pd.read_excel("C:/Users/Administrator/Desktop/2024041201.xls", header=None)

In [15]:
openWebsite()

Page is loaded within 10 seconds.


In [17]:
retryAfterError200 = 30
currentRowIdx = 0
# iterate through rows of the dataframe
# for index, row in df.iterrows():

# for i in range(len(df)):
while currentRowIdx < len(df):
    retryUntickFilter = 5
    gotResult = False
    
    # get the content of first cell 
    # brand = row[0]
    brand = str(df.loc[currentRowIdx,0])
    print("start searching brand: " + str(currentRowIdx+1) + " " + str(brand))
    try:
        searchbar = driver.find_element(By.ID, "searchbar")
        searchbar.clear()
        time.sleep(1)
        searchbar.send_keys(brand)
        time.sleep(1)
        searchbar.send_keys(Keys.ENTER)
        time.sleep(1)
    # untick filter 
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "statusDead")))
        while (driver.find_element(By.ID, "statusDead").get_attribute("checked")) and  (retryUntickFilter > 0):
            driver.find_element(By.ID, "statusDead").click()
            WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.CLASS_NAME, "spinner-border")))
            time.sleep(2)
            retryUntickFilter -= 1
    except (TimeoutException, NoSuchElementException) as e:
        print(e)
        print("           timeout or  NoSuchElementException for filter tatusDead to be clickable")
        get_url = driver.current_url
        if 'errors' in get_url:
            print('           Error page happen')
        if retryAfterError200 > 30:
            print('           Tried too much, stopped searching at row: ' + str(currentRowIdx+1) + " with name: "+ str(brand))
            break
        retryAfterError200 += 1
        driver.close()
        sleep(randint(180,300))
        openWebsite()
        continue
    
    if(not driver.find_element(By.ID, "statusDead").get_attribute("checked")):
        print("  \u2713 \u2713 \u2713 "+ "filter closed")
    else:
        print("  x x x "+ " filter NOT closed")
        
    try:
        noResult = driver.find_element(By.XPATH, "//*[text()='No results found']");
        df.loc[currentRowIdx,1] = "No results found"
        df.loc[currentRowIdx,2] = brand
        gotResult = True
        currentRowIdx += 1
        print("           ==>>> No-result-fund")
        continue
    except NoSuchElementException:
        print("           ==>>> not no-result")
    try:
        WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.CLASS_NAME, "spinner-border")))
    except TimeoutException as te:
        print(te)
        print("           timeout for spinner-border disapper")
        if retryAfterError200 > 30:
            print('           Tried too much----> stopped searching at row: ' + str(currentRowIdx+1) + " with name: "+ str(brand))
            break
        retryAfterError200 += 1
        driver.close()
        sleep(randint(180,300))
        openWebsite()
        continue
        
    try:
        resultsRow = driver.find_element(By.ID, "resultsRow")
        listResult = resultsRow.find_elements(By.CLASS_NAME, "card")
        
        nopending = False
        for card in listResult:
            try:
                badges = card.find_element(By.CLASS_NAME,"badge-light")
                print("           ==>>> Pending")
            except:
                print("           ==>>> REGISTERED")
                nopending = True
                break
        if(not nopending):
            df.loc[currentRowIdx,1] = "Pending"
            df.loc[currentRowIdx,2] = brand
        else:
            df.loc[currentRowIdx,1] = "REGISTERED"
            df.loc[currentRowIdx,2] = "notUseful"           
        gotResult = True
        currentRowIdx += 1
    except:
        print("           ==>>> No cards fund")
    if (not gotResult and retryAfterError200 <= 30):
        # handle Error Cdoe:200
        print("           ==>>> Didn't get result " + "retry remainng: " + str(30 - retryAfterError200) + "time")
        retryAfterError200 += 1
        driver.close()
        sleep(randint(180,300))
        openWebsite()
        continue
    elif (not gotResult and retryAfterError200 > 30):
        print("ERROR can't continue for brand: " +  brand + "at row: " + str(i))
        break
      # wait several seconds before the next search
    sleep(randint(3, 30))
    retryAfterError200 = 0        
# Write the dataframe object 
# into csv file 
df.to_csv ("C:/Users/Administrator/Desktop/20240408_001.csv",
                  index = False, 
                  header=False)


start searching brand: 1 ZsaeSM
  ✓ ✓ ✓ filter closed
           ==>>> not no-result
           ==>>> Pending
start searching brand: 2 TwiinSisters
  ✓ ✓ ✓ filter closed
           ==>>> No-result-fund
start searching brand: 3 JanJean
  ✓ ✓ ✓ filter closed
           ==>>> No-result-fund
start searching brand: 4 YYLRUI
  ✓ ✓ ✓ filter closed
           ==>>> No-result-fund
start searching brand: 5 Aoaowxrhgb
  ✓ ✓ ✓ filter closed
           ==>>> not no-result
           ==>>> Pending
start searching brand: 6 JovenPara
  ✓ ✓ ✓ filter closed
           ==>>> not no-result
           ==>>> Pending
start searching brand: 7 Czanzd
  ✓ ✓ ✓ filter closed
           ==>>> not no-result
           ==>>> REGISTERED
start searching brand: 8 Zelolica
  ✓ ✓ ✓ filter closed
           ==>>> not no-result
           ==>>> Pending
start searching brand: 9 TANTORA
  ✓ ✓ ✓ filter closed
           ==>>> No-result-fund
start searching brand: 10 Oulivey
  ✓ ✓ ✓ filter closed
           ==>>> not no-result

KeyboardInterrupt: 